no sys args version, for debug purpose

Following block are complete process for running KMC on API=1, from generating input files to run the kernel, Always run it see if it works after make modification on code.


following block is for generating the local cluster expansion model. See the Table S3 in support information of nasicon kmc as reference

The localclusterexpansion.initialization is for generating the **lce.json and lcesite.json**

In [ ]:
from kmcpy.model import LocalClusterExpansion

a=LocalClusterExpansion()
a.initialization(center_Na1_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a.to_json("lce.json")

the kmc.event_generator is for generating the **event_kernal.csv and events.json**

read the primitive cell from prim.json, and write events.json as output

In [ ]:
from kmcpy.event_generator import generate_events

generate_events(prim_fname=r"../examples/inputs/prim.json",supercell_shape=[2,1,1],event_fname="./events.json",event_kernal_fname='./event_kernal.csv')

Following block run the kmc version 1

In [ ]:


from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=1,**kwargs):
    os.chdir("../examples")
    inputset=InputSet.from_json("./test_input.json")

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    if api==1:


        inputset.set_parameter("occ",load_occ(inputset._parameters["mc_results"],inputset._parameters["supercell_shape"],api=inputset.api))
        # step 1 initialize global occupation and conditions
        kmc = KMC()
        events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

        # # step 2 compute the site kernal (used for kmc run)
        kmc.load_site_event_list(inputset._parameters["event_kernel"])

        # # step 3 run kmc
        kmc.run_from_database(events=events_initialized,**inputset._parameters)
    if api>1:
        raise NotImplementedError
    pass


if __name__ == "__main__":

    main()


Api version 2 initiated
aiming for more flexible inputs.
Following block is the main KMC function
DO make sure that they provide similar result as in API=1
KMC result is not necessarily the same, since it is randomized

In [ ]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=2,**kwargs):
    inputset=InputSet.from_json("test_input_v2.json",api=2)

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    inputset.set_parameter("occ",load_occ(fanme=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
    
    
    

    # step 1 initialize global occupation and conditions
    
    
    
    kmc = KMC(api=api)
    events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

    # # step 2 compute the site kernal (used for kmc run)
    kmc.load_site_event_list(inputset._parameters["event_kernel"])

    # # step 3 run kmc
    kmc.run_from_database(events=events_initialized,**inputset._parameters)


if __name__ == "__main__":

    main()

testing the sites_and_lattices.py

In [ ]:
from kmcpy.sites_and_lattices import PrimitiveCell,Site,get_rotation_matrices_from_pymatgen

get_rotation_matrices_from_pymatgen()

primitivecell=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen(),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])])

for site in primitivecell.sites:
    print(site.tag,site.relative_coordinate_in_cell,site.wyckoff_sequence)

In [ ]:
from kmcpy.sites_and_lattices import PrimitiveCell,Site,get_rotation_matrices_from_pymatgen

primitivecell=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen(),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])])

for j in primitivecell.sites:
    print(j.tag,j.relative_coordinate_in_cell,j.wyckoff_sequence)


from pymatgen.core import Structure
from pymatgen.analysis.local_env import CutOffDictNN
import numpy as np
structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")
print(structure)

local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4}
local_env_finder = CutOffDictNN(local_env_cutoff_dict)
local_env_info_list =local_env_finder.get_nn_info(structure,1)
for i in local_env_info_list:
    print(i["site"],i["site"].frac_coords,np.array(i["site"].image),i["site"].frac_coords-np.floor(i["site"].frac_coords))
    for j in primitivecell.sites:
        
        if np.allclose(i["site"].frac_coords-np.floor(i["site"].frac_coords),np.array(j.relative_coordinate_in_cell)):
            print("found corresponding",j.tag,j.wyckoff_sequence)
            break
local_env_info_list =local_env_finder.get_nn_info(structure,2)
for i in local_env_info_list:
    print(i["site"],i["site"].frac_coords,np.array(i["site"].image),i["site"].frac_coords-np.floor(i["site"].frac_coords))
    print(primitivecell.what_is_it_at(i["site"].frac_coords-np.floor(i["site"].frac_coords),raise_error=True))
 
        #print("not found")
        
        


In [ ]:
from kmcpy.sites_and_lattices import Supercell

analyzer=Supercell()

print(analyzer.structure[0].frac_coords)

for site in analyzer.reference_structure.sites:
    print(site.relative_coordinate_in_cell,site.tag,site.wyckoff_sequence)

print(analyzer.find(index_of_center_atom_of_pymatgen_structure=0))

In [ ]:
# understanding event_generator

from kmcpy.event import Event
import numpy as np

from kmcpy.io import convert
def generate_events2(prim_fname="prim.json",supercell_shape=[2,1,1],event_fname="events.json",event_kernal_fname='event_kernal.csv'):
    import json
    from kmcpy.external.pymatgen_structure import Structure
    from pymatgen.analysis.local_env import CutOffDictNN
    from pymatgen.core.lattice import Lattice
    import multiprocessing
    from joblib import Parallel, delayed
    
    print('Generating events ...')
    shape = supercell_shape
    
    print('Initializing model with pirm.json at',prim_fname,'...')
    with open(prim_fname,'r') as f:
        prim = json.load(f)
        prim_coords = [site['coordinate'] for site in prim['basis']]
        prim_specie_cases = [site['occupant_dof'] for site in prim['basis']]
        prim_lattice = Lattice(prim['lattice_vectors'])
        prim_species = [s[0] for s in prim_specie_cases]# need to change: now from [Na,Li] it will choose Na
        
    supercell_shape_matrix = np.diag(supercell_shape)
    print('Supercell Shape:\n',supercell_shape_matrix)
    structure = Structure(prim_lattice,prim_species,prim_coords)
    print('Converting supercell ...')
    structure.remove_species(['Zr','O'])
    structure.make_supercell(supercell_shape_matrix)
    structure.to(fmt='cif',filename='supercell.cif')
    # np.savetxt('frac_coords.txt',np.array(structure.frac_coords))
    # np.savetxt('cart_coords.txt',np.array(structure.cart_coords))
    # print(structure.lattice.matrix)
    # np.savetxt('latt.txt',np.array(structure.lattice.matrix))
    
    # change the parameter name to center_site (center_atom). 
    # this information  can be loaded from input (future feature)
    # currently, the sodium1 is the first two sites in prim.json
    # need to generalize to arbitrary sequence
    # future code: the input shall be a list, telling the index of center atom / Na1
    # current value 2, equivalent [0,1] (len([0,1])=2)
    #  future input [0,1] or any indices
    
    n_na1_per_prim = 2# number of Na1 per primitive cell
    
    
    n_na1 = n_na1_per_prim*np.prod(shape) # total number of Na1 in the supercell. This should be equal to the empty lines in the event_kernal.csv
    
    
    local_env_cutoff_dict = {('Na','Na'):4,('Na','Si'):4}
    ncpus = multiprocessing.cpu_count()
    print('Looking for all possible swaps with ncpus =',ncpus,' (this might take a while) ...')
    na_1_sites = structure[0:n_na1]
    local_env_finder = CutOffDictNN(local_env_cutoff_dict)
    
    # get_nn_info; return the neiarest neighbor (or local environment)
    local_env_info_list = list(Parallel(n_jobs=ncpus)(delayed(local_env_finder.get_nn_info)(structure,i) for i in np.arange(0,n_na1)))
    # change to scipy.spatial.cKDTtree in the future
    # get_nn_info return list of these thing:
    # local_env_info_list[0]: local environmental information of 0th Na1
    # local_env_info_list[0] includes (or get_nn_info return) list of neighbors, with each neighbor as dictionary of: {site, image(periodic structure index),weight,site_index}     
    
    events = []
    events_dict = []
    for (na1_index,(na1_site,local_env_info)) in enumerate(zip(na_1_sites,local_env_info_list)):
        for local_env in local_env_info:
            # only consider the environment of Na, but not S or P.
            # need generalization for other system.
            # "Na" -> specie_to_diffuse
            # need to be input.
            # probably become a list.
            if "Na" in local_env['site'].specie.symbol:
                this_event = Event()
                this_event.initialization(na1_index,local_env['site_index'],local_env_info)
                events.append(this_event)
                events_dict.append(this_event.as_dict())

In [ ]:
from kmcpy.sites_and_lattices import Supercell,PrimitiveCell,get_rotation_matrices_from_pymatgen,Site
from pymatgen.core import Structure


analyzer=Supercell(original_structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"), local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},reference_structure=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])]),verbose=False)


#print(analyzer.find(index_of_center_atom_of_pymatgen_structure=0))

analyzer.create_supercell(indices_of_center_atom=[0,1,2,3,4,5],center_atom_tag="Na1",diffuse_to="Na2",environment=["Na2","Si"],supercell_shape=[1,1,2],event_fname="events.json",event_kernal_fname='event_kernal.csv')

In [ ]:
import json
from pymatgen.core import Structure,Lattice

prim_fname="/Users/weihangxie/Documents/GitHub/kmcPy/dev_test/prim.json"
with open(prim_fname,'r') as f:
    prim = json.load(f)
    prim_coords = [site['coordinate'] for site in prim['basis']]
    prim_specie_cases = [site['occupant_dof'] for site in prim['basis']]
    prim_lattice = Lattice(prim['lattice_vectors'])
    prim_species = [s[0] for s in prim_specie_cases]


structure = Structure(prim_lattice,prim_species,prim_coords)
structure.to("cif","primitive.cif")

我写的site lattice太智障了,看不懂.


根据piero建议,还是用structure写吧.....

In [ ]:
from kmcpy.external.pymatgen_structure import Structure
from pymatgen.symmetry.analyzer  import SpacegroupAnalyzer
from kmcpy.external.pymatgen_local_env import CutOffDictNN
structure=Structure.from_cif("/Users/weihangxie/Documents/GitHub/kmcPy/dev_test/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")



#print(structure.site_properties)
print(structure)

#print(structure[5].properties)

structure.make_supercell(scaling_matrix=[1,1,2])
#print(structure.site_properties)
local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4}
local_env_finder = CutOffDictNN(local_env_cutoff_dict)
local_env_finder.get_nn_info(structure,0)   
from kmcpy.model import LocalClusterExpansion



a=LocalClusterExpansion(api=1)
a.initialization(center_Na1_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a=LocalClusterExpansion(api=2)
a.initialization(center_atom_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a.to_json("test_LCEv2.json")


test on find_site_by_wyckoff_sequence_and_label

In [ ]:
from kmcpy.external.pymatgen_structure import Structure

        
structure=Structure.from_cif("/Users/weihangxie/Documents/GitHub/kmcPy/dev_test/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")

site1=structure.find_site_by_wyckoff_sequence_and_label(wyckoff_sequence=1,label="Na1")
site2=structure.find_site_by_wyckoff_sequence_and_label(wyckoff_sequence=9,label="Na2")
print(site1,site2,"distance is",site1.distance(site2))

structure.make_supercell([2,3,1])
print(structure)

test on nd supercell

In [1]:
import numpy as np
scaling_matrix=[1,2,3]
scale_matrix = np.array(scaling_matrix, int)
for dim1,dim2,dim3 in np.ndindex((1,2,3)):
    print(dim1, dim2, dim3)
    
np.dot(np.array([1,2,3]),np.array((2,5,7)))

from kmcpy.external.pymatgen_structure import Structure
structure=Structure.from_cif("/Users/weihangxie/Documents/GitHub/kmcPy/dev_test/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")
supercell=structure.make_kmc_supercell(scaling_matrix=(1,2,4))

print(supercell)


np.array([1,2,3])-np.array([4,23,12])

0 0 0
0 0 1
0 0 2
0 1 0
0 1 1
0 1 2
{(0.0, 0.0, 0.14719): Comp: Zr1, (0.0, 0.0, 0.0): Comp: Na1, (-0.36033, 0.0, 0.25): Comp: Na1, (0.18534, 0.16651, 0.08652): Comp: O1, (0.18214, 0.02102, 0.19199): Comp: O1, (0.29544, 0.0, 0.25): Comp: Si1}
Full Formula (Na192 Zr96 Si144 O576)
Reduced Formula: Na4Zr2(SiO4)3
abc   :   9.186300  18.372600  88.724000
angles:  90.000000  90.000000 120.000000
Sites (1008)
   #  SP           a         b         c  label      local_index  supercell      wyckoff_sequence
----  ----  --------  --------  --------  -------  -------------  -----------  ------------------
   0  Na+   0         0         0.125     Na1                 12  (0, 0, 0)                     0
   1  Na+   0         0         0.375     Na1                 12  (0, 0, 1)                     0
   2  Na+   0         0         0.625     Na1                 12  (0, 0, 2)                     0
   3  Na+   0         0         0.875     Na1                 12  (0, 0, 3)                     0
   4  N

array([ -3, -21,  -9])

test the new event generator

Problem!!!!! LOok the distance matrix, they are not all the same

In [1]:
from kmcpy.event_generator import generate_events2
from kmcpy.external.pymatgen_structure import Structure

primitive_structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)
primitive_structure.to("cif","primitive_EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")


#generate_events2(prim_cif_name="primitive_EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,3,4],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',center_atom_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True)


#generate_events2(prim_cif_name="EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,3,4],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',center_atom_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True,hacking_arg={1: [27, 29, 28, 30, 32, 31, 117, 119, 118, 120, 122, 121],2:[21,22,23,32,30,31,111,112,113,122,120,121],3:[18,20,19,34,33,35,108,110,109,124,123,125],5:[21,23,22,24,26,25,111,113,112,114,116,115]})
generate_events2(prim_cif_name="EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,3,4],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',center_atom_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True,hacking_arg={2:[21,22,23,32,30,31,111,112,113,122,120,121],3:[18,20,19,34,33,35,108,110,109,124,123,125],5:[21,23,22,24,26,25,111,113,112,114,116,115]})


{(0.0, 0.0, 0.14719): Comp: Zr1, (0.0, 0.0, 0.0): Comp: Na1, (-0.36033, 0.0, 0.25): Comp: Na1, (0.18534, 0.16651, 0.08652): Comp: O1, (0.18214, 0.02102, 0.19199): Comp: O1, (0.29544, 0.0, 0.25): Comp: Si1}
Generating events ...
Initializing model with input cif file at EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif ...
{(0.0, 0.0, 0.14719): Comp: Zr1, (0.0, 0.0, 0.0): Comp: Na1, (-0.36033, 0.0, 0.25): Comp: Na1, (0.18534, 0.16651, 0.08652): Comp: O1, (0.18214, 0.02102, 0.19199): Comp: O1, (0.29544, 0.0, 0.25): Comp: Si1}


-------------------------------------------


receive string type in 'center atom label' parameter. Trying to find the indices of center atom
please check if this is a center atom: [ 0.      0.     11.0905] Na+ {'wyckoff_sequence': 0, 'local_index': 12, 'label': 'Na1'}
please check if this is a center atom: [0. 0. 0.] Na+ {'wyckoff_sequence': 1, 'local_index': 13, 'label': 'Na1'}
please check if this is a center atom: [ 4.59315     2.65185639 18.48416667] Na+ {'wyckoff